In [29]:
import torch
import torchvision
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
import os 
from warnings import simplefilter
import pandas as pd
from imblearn.over_sampling import SMOTE  


In [30]:
import models
import class_sampling
import train
import metric_utils
import inference
import loss_fns
import torchvision.ops 

In [31]:
NUM_CLASSES = 10
n_epochs = 30
batch_size_train = 64
batch_size_test = 1000
momentum = 0

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

NUM_CLASSES_REDUCED = 2
nums = (0, 1)
ratio = (10, 1)

CLASS_LABELS = {'airplane': 0,
                 'automobile': 1,
                 'bird': 2,
                 'cat': 3,
                 'deer': 4,
                 'dog': 5,
                 'frog': 6,
                 'horse': 7,
                 'ship': 8,
                 'truck': 9}


simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=UserWarning)
simplefilter(action='ignore', category=DeprecationWarning)

In [32]:
col_names = ["name", 
            "num_classes", 
            "classes_used", 
            "ratio", 
            "learning_rate", 
            "mean_0", "variance_0",
            "mean_10", "variance_10",
            "mean_20", "variance_20",
            "mean_30", "variance_30",
            "mean_40", "variance_40",
            "mean_50", "variance_50"]

rows = []

In [33]:
train_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor() ]))  


test_CIFAR10 = torchvision.datasets.CIFAR10('cifar10', train=False, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor()]))

train_CIFAR10.data = train_CIFAR10.data.reshape(50000, 3, 32, 32)
test_CIFAR10.data = test_CIFAR10.data.reshape(10000, 3, 32, 32)

    
reduced_train_CIFAR10 = class_sampling.Reduce(train_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True)
reduced_test_CIFAR10 = class_sampling.Reduce(test_CIFAR10, NUM_CLASSES_REDUCED, nums=nums, CIFAR=True)

ratio_train_CIFAR10 = class_sampling.Ratio(train_CIFAR10, NUM_CLASSES_REDUCED, ratio, nums=nums)

smote_train_CIFAR10 = class_sampling.Smote(ratio_train_CIFAR10)

Files already downloaded and verified
Files already downloaded and verified


In [34]:
targets = ratio_train_CIFAR10.labels 

class_count = np.unique(targets, return_counts=True)[1]
print(class_count)

weight = 1. / class_count

samples_weight = weight[targets]
samples_weight = torch.from_numpy(samples_weight)
oversampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(max(class_count) * NUM_CLASSES_REDUCED), replacement=True)
sampler = torch.utils.data.WeightedRandomSampler(samples_weight, len(samples_weight), replacement=True)
undersampler = torch.utils.data.WeightedRandomSampler(samples_weight, int(min(class_count) * NUM_CLASSES_REDUCED), replacement=False)

weight *= class_count[0]

[5000  500]


In [35]:
beta = 0.999 

exp = np.empty_like(targets)
for i, count in enumerate(class_count):
    exp[targets==i] = count
effective_weights = (1 - beta) / ( 1 - (beta ** torch.from_numpy(exp)))

In [36]:
train_loader_reduced = DataLoader(reduced_train_CIFAR10, batch_size=batch_size_train, shuffle=True)  

train_loader_ratio = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, shuffle=True) 

train_loader_oversampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=oversampler)

train_loader_undersampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=undersampler)

train_loader_sampled = DataLoader(ratio_train_CIFAR10, batch_size=batch_size_train, sampler=sampler)

train_loader_smote = DataLoader(smote_train_CIFAR10, batch_size=batch_size_train, shuffle=True)

test_loader_reduced = DataLoader(reduced_test_CIFAR10, batch_size=batch_size_test, shuffle=True)

In [37]:
# 2 CLASS normal

momentum=0
learning_rates = [1e-4, 1e-5]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_reduced, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["normal", 2, nums, (1, 1), learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.001968830943107605, AUC: 0.6361665000000001


Test set: Avg. loss: 0.0006069920063018799, AUC: 0.8331379999999999


Test set: Avg. loss: 0.0005906819999217987, AUC: 0.849458


Test set: Avg. loss: 0.0005717484056949615, AUC: 0.8692640000000001


Test set: Avg. loss: 0.0032240989208221437, AUC: 0.38664699999999996


Test set: Avg. loss: 0.0006730922758579254, AUC: 0.7028579999999999


Test set: Avg. loss: 0.000657876968383789, AUC: 0.7713125000000001


Test set: Avg. loss: 0.0006426167488098144, AUC: 0.808087


Test set: Avg. loss: 0.006873347997665405, AUC: 0.683021


Test set: Avg. loss: 0.0006474375724792481, AUC: 0.7793890000000001


Test set: Avg. loss: 0.0006305342614650727, AUC: 0.8076369999999999


Test set: Avg. loss: 0.000615574598312378, AUC: 0.8379340000000001


Test set: Avg. loss: 0.001451125681400299, AUC: 0.47390000000000004


Test set: Avg. loss: 0.0006873179376125335, AUC: 0.5776140000000001


Test set: Avg. loss: 0.0006742987930774689, AUC: 0.6

In [38]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])


df2.to_csv('results/convnet_aucs.csv', index=False)

rows = []


In [39]:
# 2 CLASS ratio

momentum=0
learning_rates = [1e-4, 1e-5]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["ratio", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.0019287337064743042, AUC: 0.518386


Test set: Avg. loss: 0.0008795691728591919, AUC: 0.725559


Test set: Avg. loss: 0.0007942625284194946, AUC: 0.7863410000000001


Test set: Avg. loss: 0.0007645726501941681, AUC: 0.8171459999999999


Test set: Avg. loss: 0.0018440639972686767, AUC: 0.5057195000000001


Test set: Avg. loss: 0.0009616871476173401, AUC: 0.647203


Test set: Avg. loss: 0.0008821539282798767, AUC: 0.760052


Test set: Avg. loss: 0.0008314119577407837, AUC: 0.798547


Test set: Avg. loss: 0.0011216624975204468, AUC: 0.4446


Test set: Avg. loss: 0.001142448902130127, AUC: 0.577028


Test set: Avg. loss: 0.0009208276271820069, AUC: 0.749444


Test set: Avg. loss: 0.000837091475725174, AUC: 0.7970310000000002


Test set: Avg. loss: 0.0019242210388183594, AUC: 0.582731


Test set: Avg. loss: 0.00100971981883049, AUC: 0.71405


Test set: Avg. loss: 0.0009351155459880829, AUC: 0.8056760000000001


Test set: Avg. loss: 0.0008753396570682526, AUC: 0.84036

In [40]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [41]:
# 2 CLASS oversampled

momentum=0
learning_rates = [1e-4, 1e-5]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_oversampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["oversampled", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)




Test set: Avg. loss: 0.0048054039478302, AUC: 0.551255


Test set: Avg. loss: 0.0006570699214935303, AUC: 0.7241254999999999


Test set: Avg. loss: 0.0006405693292617797, AUC: 0.7878685000000001


Test set: Avg. loss: 0.0006275400221347809, AUC: 0.816168


Test set: Avg. loss: 0.0015801175236701966, AUC: 0.7138960000000001


Test set: Avg. loss: 0.0006501563191413879, AUC: 0.7620300000000002


Test set: Avg. loss: 0.0006346423923969268, AUC: 0.8019250000000002


Test set: Avg. loss: 0.0006165396571159362, AUC: 0.8341529999999999


Test set: Avg. loss: 0.0012231424450874328, AUC: 0.462604


Test set: Avg. loss: 0.0006932708323001862, AUC: 0.4976065


Test set: Avg. loss: 0.0006929941773414612, AUC: 0.519285


Test set: Avg. loss: 0.000692833960056305, AUC: 0.505828


Test set: Avg. loss: 0.002317346930503845, AUC: 0.502827


Test set: Avg. loss: 0.0006949883997440338, AUC: 0.505665


Test set: Avg. loss: 0.0006948157250881195, AUC: 0.5026390000000001


Test set: Avg. loss: 0.0006946371

In [42]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [43]:
# 2 CLASS undersampled

momentum=0
learning_rates = [1e-4, 1e-5]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_undersampled, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["undersampled", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.001361272394657135, AUC: 0.5182420000000001


Test set: Avg. loss: 0.0006831247210502625, AUC: 0.6231269999999999


Test set: Avg. loss: 0.0006858513355255127, AUC: 0.6093769999999998


Test set: Avg. loss: 0.0006839762032032013, AUC: 0.6354010000000003


Test set: Avg. loss: 0.0014974244236946107, AUC: 0.397626


Test set: Avg. loss: 0.0006841929852962493, AUC: 0.634284


Test set: Avg. loss: 0.000684006541967392, AUC: 0.635391


Test set: Avg. loss: 0.0006836210787296295, AUC: 0.6420640000000002


Test set: Avg. loss: 0.0029681077003479005, AUC: 0.438403


Test set: Avg. loss: 0.0006588656306266785, AUC: 0.7569130000000001


Test set: Avg. loss: 0.0006624727547168732, AUC: 0.750571


Test set: Avg. loss: 0.0006612907946109772, AUC: 0.754692


Test set: Avg. loss: 0.0009737875759601593, AUC: 0.42999649999999995


Test set: Avg. loss: 0.0006970759928226471, AUC: 0.4916975


Test set: Avg. loss: 0.0006920590996742249, AUC: 0.5131445


Test set: Avg. loss: 0.00068

In [44]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [45]:
# 2 CLASS SMOTE

momentum=0
learning_rates = [1e-4, 1e-5]

learning_rate_aucs = []


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_smote, network, optimizer, verbose=False)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["smote", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.00392682933807373, AUC: 0.608166


Test set: Avg. loss: 0.0006436906456947326, AUC: 0.7887475


Test set: Avg. loss: 0.0006319799125194549, AUC: 0.8360875000000001


Test set: Avg. loss: 0.0006192918419837952, AUC: 0.8520330000000003


Test set: Avg. loss: 0.0008545547127723694, AUC: 0.590994


Test set: Avg. loss: 0.0006431725919246674, AUC: 0.780115


Test set: Avg. loss: 0.0006292569637298584, AUC: 0.8135020000000001


Test set: Avg. loss: 0.0006201825439929962, AUC: 0.8270049999999999


Test set: Avg. loss: 0.013260830402374267, AUC: 0.45235649999999994


Test set: Avg. loss: 0.0006229948401451111, AUC: 0.803364


Test set: Avg. loss: 0.0006227455139160156, AUC: 0.8083864999999998


Test set: Avg. loss: 0.000606639415025711, AUC: 0.822162


Test set: Avg. loss: 0.0028494004011154176, AUC: 0.497351


Test set: Avg. loss: 0.0006307647228240967, AUC: 0.7854829999999999


Test set: Avg. loss: 0.0006063108146190644, AUC: 0.823493


Test set: Avg. loss: 0.00059516

In [46]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

rows = []

In [47]:
# 2 CLASS Focal Loss

momentum=0
learning_rates = [1e-4, 1e-5]

learning_rate_aucs = []

loss_fn_args = {}
loss_fn_args['reduction'] = 'mean'


for learning_rate in learning_rates:
    aucs = []
    for i in range(10):
        model_aucs = []
        network = models.ConvNet(NUM_CLASSES_REDUCED)
        optimizer = optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
        _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network) 
        model_aucs.append(auc)
        for epoch in range(n_epochs):
            _, _ = train.train_sigmoid(epoch, train_loader_ratio, network, optimizer, verbose=False, loss_fn=loss_fns.SigmoidFocalLoss, loss_fn_args=loss_fn_args)
            if (epoch + 1) % 10 == 0: 
                _, auc = metric_utils.auc_sigmoid(test_loader_reduced, network)
                model_aucs.append(auc)
        aucs.append(model_aucs)
    learning_rate_aucs.append(aucs)

learning_rate_aucs = np.asarray(learning_rate_aucs)

auc_mean = np.mean(learning_rate_aucs, axis=1)
auc_variance = np.var(learning_rate_aucs, axis=1)

for i in range(len(learning_rates)): 
    row = ["focal_loss", 2, nums, ratio, learning_rates[i],
            auc_mean[i][0], auc_variance[i][0], 
            auc_mean[i][1], auc_variance[i][1],
            auc_mean[i][2], auc_variance[i][2],
            auc_mean[i][3], auc_variance[i][3]]
    rows.append(row)



Test set: Avg. loss: 0.0016703126430511475, AUC: 0.502707


Test set: Avg. loss: 0.0008640770018100739, AUC: 0.4535


Test set: Avg. loss: 0.0008134927153587341, AUC: 0.5995950000000001


Test set: Avg. loss: 0.000739108681678772, AUC: 0.6680075


Test set: Avg. loss: 0.00121213698387146, AUC: 0.44127250000000007


Test set: Avg. loss: 0.0008791627585887909, AUC: 0.427161


Test set: Avg. loss: 0.0008808626532554627, AUC: 0.4839875


Test set: Avg. loss: 0.0008660988509654999, AUC: 0.5037349999999999


Test set: Avg. loss: 0.0019614633321762085, AUC: 0.47862899999999997


Test set: Avg. loss: 0.0009047038555145264, AUC: 0.42100249999999995


Test set: Avg. loss: 0.0008650421500205994, AUC: 0.4492385


Test set: Avg. loss: 0.0008351796269416809, AUC: 0.48053500000000005


Test set: Avg. loss: 0.0024454493522644044, AUC: 0.46927300000000005


Test set: Avg. loss: 0.0008800350725650788, AUC: 0.52539


Test set: Avg. loss: 0.0007984228134155273, AUC: 0.549548


Test set: Avg. loss: 0.0007

In [48]:
df1 = pd.read_csv('results/convnet_aucs.csv')

df2 = pd.DataFrame(rows, columns = col_names[0:13]) 

df = pd.concat([df1, df2])

df.to_csv('results/convnet_aucs.csv', index=False)

In [49]:
df2.head(50)

,name,num_classes,classes_used,ratio,learning_rate,mean_0,variance_0,mean_10,variance_10,mean_20,variance_20,mean_30,variance_30
0,focal_loss,2,"(0, 1)","(10, 1)",0.00010,0.486575,0.005565,0.477467,0.007103,0.530767,0.006378,0.565553,0.006784
1,focal_loss,2,"(0, 1)","(10, 1)",0.00001,0.537682,0.012789,0.538864,0.005410,0.474367,0.008302,0.458953,0.007102
